In [3]:
 conda env create -f /Users/rratnappan/Documents/Confera/Hamilton_worklist/notalims.yml


Note: you may need to restart the kernel to use updated packages.



CondaValueError: prefix already exists: C:\Users\rratnappan\Anaconda3\envs\notalims



In [12]:
import yaml
import pandas as pd

In [13]:
conda activate notalims


Note: you may need to restart the kernel to use updated packages.


In [14]:
python notnotalims.py

SyntaxError: invalid syntax (4132950207.py, line 1)

In [1]:
# %load C:/Users/rratnappan/Documents/Confera/Hamilton_worklist/notnotalims.py
import os
import sys
import pandas as pd
from math import sqrt
import numpy as np
import getpass
import string
from PyQt5 import QtCore, QtGui, QtWidgets

global script_vers
script_vers = 'v2.1'


class hc_norm:
    def __init__(
        self,
        target_mass,
        vol_remain,
        baitset
    ):
        self.target_mass = float(target_mass)  # target mass input to HC in ng
        self.vol_remain = float(vol_remain)  # LC volume remaining in μL
        self.baitset = str(baitset)  # name of baitset used


class mpl:
    def __init__(
        self,
        final_conc,
        pool_vol,
        avg_frag_length,
        sample_draw_vol
    ):
        self.final_conc = float(final_conc)  # final pool conc in nM
        self.pool_vol = float(pool_vol)  # final pooling volume in μL
        self.avg_frag_length = float(avg_frag_length)  # average fragment size for nM conversion (bp)
        self.sample_draw_vol = float(sample_draw_vol)  # vol of sample to draw, μL (default = 10)


def get_column(input_well):
    if len(input_well) == 3:
        return int(input_well[-2:])
    else:
        return int(input_well[-1:])


def well_order():
    cols = range(1, 13)  # 1 -> 12
    rows = [i for i in string.ascii_uppercase[:8]]  # A -> H
    wells = []
    for col in cols:
        for row in rows:
            wells.append(f'{row}{col}')
    return wells


def process_hc_norm(hc_norm_obj, lc_plate_name, input_data):

    max_well_vol = 180  # max allowed vol per well (shouldn't ever exceed this)

    worklist_columns = [
        'SourcePlate',
        'Sample',
        'Source',
        'SWell',
        'DNA',
        'Destination',
        'DWell',
        'BaitSet'
    ]

    hc_worklist = pd.DataFrame(columns=worklist_columns)

    wells = well_order()

    well_index = 0
    for index, row in input_data.iterrows():
        lc_conc = float(row['Conc'])  # lc conc (ng/μL)
        source_well = row['Well']  # source well
        target_mass = hc_norm_obj.target_mass  # target mass (ng)
        vol_remain = hc_norm_obj.vol_remain - 2.5  # remaining LC vol (plus wiggle room, μL)
        if vol_remain < 0:  # no negatives!
            vol_remain = 0

        sample_vol = round(target_mass / lc_conc, 2)
        if sample_vol > vol_remain:
            sample_vol = vol_remain
        if sample_vol > max_well_vol:
            sample_vol = max_well_vol

        hc_worklist.at[index, 'Source'] = 'DNAPlate'  # static
        hc_worklist.at[index, 'Destination'] = 'PoolPlate'  # static

        hc_worklist.at[index, 'SourcePlate'] = lc_plate_name  # same source plate for all
        hc_worklist.at[index, 'Sample'] = f'{lc_plate_name}_{source_well}'  # generic name
        hc_worklist.at[index, 'SWell'] = source_well  # source well for sample
        hc_worklist.at[index, 'DNA'] = sample_vol  # vol of sample to add
        hc_worklist.at[index, 'DWell'] = wells[well_index]  # remap to new well position, go in order
        hc_worklist.at[index, 'BaitSet'] = hc_norm_obj.baitset  # baitset to use

        well_index += 1  # iterate to next well index

    return hc_worklist


# function to check if proper dilutions are being made
# based on MPL input config - warns user of deviations
def hc_validate_input(hc_norm_obj, input_data, output_worklist):

    source_plate = output_worklist['SourcePlate'].unique().item()
    output_plate = source_plate.replace('LC', 'HC')

    norm_params = [
        "HC normalization parameters:",
        f"\tinput plate name: {source_plate}",
        f"\toutput plate name: {output_plate}",
        f"\ttarget mass input: {hc_norm_obj.target_mass} ng",
        f"\tremaining LC volume: {hc_norm_obj.vol_remain} uL",
        f"\tbaitset used: {hc_norm_obj.baitset}"
    ]

    log_warnings = []
    log_detail = []

    for index, row in input_data.iterrows():

        sample_conc = float(row['Conc'])  # sample conc
        source_well = row['Well']  # sample well
        norm_vol = output_worklist[output_worklist['SWell'] == source_well]['DNA'].item()  # normalized sample vol
        norm_mass = round(sample_conc * norm_vol, 0)  # how much mass is being taken

        if norm_mass < hc_norm_obj.target_mass:
            log_warnings.append(
                f'! warning: {source_well} normalized mass is {norm_mass} ng'
            )

        log_line = f'{source_well} @ {sample_conc} ng/uL -> {norm_mass} ng HC input, {norm_vol} uL used'
        log_detail.append(log_line)

    process_log = []  # will be a summary of log data captured above
    for item in norm_params:
        process_log.append(item)
    process_log.append(' ')  # blank line
    for item in log_warnings:
        process_log.append(item)
    process_log.append(' ')  # blank line
    for item in log_detail:
        process_log.append(item)

    return process_log


def process_mpl(mpl_obj, hc_plate_name, input_data):

    max_well_vol = 180  # max allowed volume per single well/dilution (PCR plate)
    max_tube_vol = 1300  # max allowed volume per MPL tube (1.5 mL eppendorf)
    sample_draw_vol = mpl_obj.sample_draw_vol  # draw volume (e.g., 10 μL)
    # prevent total pool vol from overwhemling labware (~13.5 μL for 96 sample plate)
    # still allows for 310 μL * 4, or enough volume to run four full S4 FCs
    n_samples = len(input_data)
    if n_samples * mpl_obj.pool_vol >= max_tube_vol:
        mpl_obj.pool_vol = max_tube_vol / n_samples

    source_plate = hc_plate_name  # name of HC plate (user provided)

    # fill in worklist columns (from noatlims template)
    worklist_columns = [
        'SourcePlate',
        'SourceAliquot',
        'EBSource',
        'EBWell',
        'DilutionPlate',
        'SourceWell',
        'DilutionWell',
        'EBLittle',
        'EBBig',
        'DNASource',
        'DNA1Vol',
        'DNA2Vol',
        'PoolSource',
        'PoolDestination',
        'PoolWell',
        'PoolVolume'
    ]

    # will make two "worklists", primary and secondary dilution
    mpl_worklist_dil1 = pd.DataFrame(columns=worklist_columns)
    mpl_worklist_dil2 = pd.DataFrame(columns=worklist_columns)

    for index, row in input_data.iterrows():
        hc_conc = float(row['Conc'])  # sample conc
        source_well = row['Well']  # sample well
        nm_conc = hc_conc / 660 * (10 ** 6) / mpl_obj.avg_frag_length  # convert to nM

        # calculate volume target for dilution of sample to target nM conc:
        # e.g., 2 nM (sample conc) * 10 μL (sample vol) / 1 nM (target conc) = 20 μL total
        target_1 = round((nm_conc * sample_draw_vol) / mpl_obj.final_conc, 2)
        # if less than individual sample vol
        # set target to pooling vol + 2.5 (so there's some dead volume)
        # e.g., 7.5 μL + 2.5 μL
        if target_1 < sample_draw_vol:
            target_1 = mpl_obj.pool_vol + 2.5
        # calculate secondary target vol, assuming below double-dilution cutoff:
        # e.g., 30 μL (first target) - 10 μL (sample vol) = 20 μL (secondary target)
        # if below cutoff, ignore secondary calculations
        if target_1 <= 100:
            target_2 = round(target_1 - sample_draw_vol, 2)
            if target_2 > max_well_vol:
                target_2 = max_well_vol - sample_draw_vol
            ignore = True
        # otherwise, calculate diluent for double-dilution
        else:
            diluent = sqrt((nm_conc * (sample_draw_vol ** 2)) / mpl_obj.final_conc)
            target_2 = round(diluent - sample_draw_vol, 2)
            if target_2 > max_well_vol:
                target_2 = max_well_vol - sample_draw_vol
            ignore = False

        mpl_worklist_dil1.at[index, 'DilutionPlate'] = 'DilutionPlate1'  # static
        mpl_worklist_dil1.at[index, 'DNASource'] = 'DNAPlate1'  # static
        mpl_worklist_dil1.at[index, 'EBSource'] = 'EBRes'  # static
        mpl_worklist_dil1.at[index, 'PoolDestination'] = 'PoolPlate'  # static
        mpl_worklist_dil1.at[index, 'EBWell'] = 1  # static, trough position
        mpl_worklist_dil1.at[index, 'PoolWell'] = 'A1'  # static

        mpl_worklist_dil1.at[index, 'DilutionWell'] = source_well  # dest well = source well
        mpl_worklist_dil1.at[index, 'DNA1Vol'] = sample_draw_vol  # sample vol to draw
        mpl_worklist_dil1.at[index, 'SourcePlate'] = source_plate  # source plate name
        mpl_worklist_dil1.at[index, 'SourceWell'] = source_well  # source well
        mpl_worklist_dil1.at[index, 'PoolVolume'] = mpl_obj.pool_vol  # final vol to pool

        # check if secondary target is > or < 50
        # determines if big vs little dilution is made
        # two separate dilutions, 50 μL vs 300 μL tips
        if target_2 <= 50:
            mpl_worklist_dil1.at[index, 'EBLittle'] = target_2
        else:
            mpl_worklist_dil1.at[index, 'EBLittle'] = 0

        if target_2 > 50:
            mpl_worklist_dil1.at[index, 'EBBig'] = target_2
        else:
            mpl_worklist_dil1.at[index, 'EBBig'] = 0

        if ignore is False:
            mpl_worklist_dil2.at[index, 'DNA1Vol'] = 0  # static
            mpl_worklist_dil2.at[index, 'DNA2Vol'] = 0  # static
            mpl_worklist_dil2.at[index, 'PoolVolume'] = 0  # static
            mpl_worklist_dil2.at[index, 'EBWell'] = 1  # static
            mpl_worklist_dil1.at[index, 'PoolSource'] = 'DilutionPlate2'  # static
            mpl_worklist_dil2.at[index, 'DilutionPlate'] = 'DilutionPlate2'  # static
            mpl_worklist_dil2.at[index, 'DNASource'] = 'DNAPlate1'  # static
            mpl_worklist_dil2.at[index, 'EBSource'] = 'EBRes'  # static
            mpl_worklist_dil2.at[index, 'PoolDestination'] = 'PoolPlate'  # static
            mpl_worklist_dil2.at[index, 'PoolSource'] = 'DilutionPlate1'  # static
            mpl_worklist_dil2.at[index, 'PoolWell'] = 'A1'  # static

            mpl_worklist_dil2.at[index, 'DilutionWell'] = source_well  # dest well = source well
            mpl_worklist_dil1.at[index, 'DNA2Vol'] = sample_draw_vol  # sample vol to draw
            mpl_worklist_dil2.at[index, 'SourcePlate'] = source_plate  # source plate name
            mpl_worklist_dil2.at[index, 'SourceWell'] = source_well  # source well

            if target_2 <= 50:
                mpl_worklist_dil2.at[index, 'EBLittle'] = target_2
            else:
                mpl_worklist_dil2.at[index, 'EBLittle'] = 0

            if target_2 > 50:
                mpl_worklist_dil2.at[index, 'EBBig'] = target_2
            else:
                mpl_worklist_dil2.at[index, 'EBBig'] = 0

        if ignore is True:
            mpl_worklist_dil1.at[index, 'DNA2Vol'] = 0  # static
            mpl_worklist_dil1.at[index, 'PoolSource'] = 'DilutionPlate1'  # static

    # combine dilution 1 / dilution 2
    output = pd.concat([mpl_worklist_dil1, mpl_worklist_dil2])

    # clean up and sort worklist
    output = output.sort_values(by='SourceWell', ascending=True)
    output['col'] = output['SourceWell'].apply(get_column)
    output['row'] = output['SourceWell'].apply(lambda x: x[0])
    output = output.sort_values(['col', 'row', 'DilutionPlate'], ascending=[True, True, True])
    output.drop(['col', 'row'], axis=1, inplace=True)
    output = output.replace(np.nan, '', regex=True)

    return output


# function to check if proper dilutions are being made
# based on MPL input config - warns user of deviations
def validate_mpl_dilutions(mpl_obj, input_data, output_worklist):

    mpl_params = [
        "MPL parameters:",
        f"\tinput plate name: {output_worklist['SourcePlate'].unique().item()}",
        f"\tvol of sample to dilute: {mpl_obj.sample_draw_vol} uL",
        "\tdouble-dilution cutoff: 100 uL",
        f"\tfinal target conc: {mpl_obj.final_conc} nM",
        f"\tdiluted sample to pool: {mpl_obj.pool_vol} uL",
        f"\tavg frag length (nM conversion): {mpl_obj.avg_frag_length} bp",
        f"\ttotal pool volume: {output_worklist['PoolVolume'].sum()} uL"
    ]

    log_warnings = []
    log_detail = []

    for index, row in input_data.iterrows():

        hc_conc = round(float(row['Conc']), 2)  # <- specific to this example, need to fix
        source_well = row['Well']  # <- specific to this example, need to fix
        nm_conc = round(hc_conc / 660 * (10 ** 6) / mpl_obj.avg_frag_length, 2)  # convert to nM

        row_data = [row for row in output_worklist[output_worklist['SourceWell'] == source_well].iterrows()]
        if len(row_data) > 1:
            line1 = row_data[0][1]
            line2 = row_data[1][1]

            dil1 = nm_conc * line1['DNA1Vol'] / (line1['EBLittle'] + line1['EBBig'] + line1['DNA1Vol'])
            dil2 = dil1 * line1['DNA2Vol'] / (line2['EBLittle'] + line2['EBBig'] + line1['DNA2Vol'])

            output_conc = round(dil2, 1)

        else:
            line1 = row_data[0][1]

            dil1 = nm_conc * line1['DNA1Vol'] / (line1['EBLittle'] + line1['EBBig'] + line1['DNA1Vol'])

            output_conc = round(dil1, 1)

        if output_conc < mpl_obj.final_conc:
            log_warnings.append(
                f'! warning: {source_well} final concentration is {output_conc} nM'
            )
        elif output_conc > mpl_obj.final_conc:
            log_warnings.append(
                f'! warning: {source_well} final concentration is {output_conc} nM'
            )

        log_line = f'{source_well} @ {hc_conc} ng/uL -> {nm_conc} nM, diluted to {output_conc} nM'
        log_detail.append(log_line)

    process_log = []  # will be a summary of log data captured above
    for item in mpl_params:
        process_log.append(item)
    process_log.append(' ')  # blank line
    for item in log_warnings:
        process_log.append(item)
    process_log.append(' ')  # blank line
    for item in log_detail:
        process_log.append(item)

    return process_log


# class definition for the UI object
class Ui_NOTNOTALIMS(object):

    def setupUi(self, NOTNOTALIMS):

        # custom table class (to allow for copy-pasting of data)
        class customTable(QtWidgets.QTableWidget):
            # function to handle Ctrl-V pasting into table
            def keyPressEvent(self, event):
                if event.key() == QtCore.Qt.Key_V and event.modifiers() == QtCore.Qt.ControlModifier:

                    clip = QtWidgets.QApplication.clipboard()  # clipboard contents
                    mime = clip.mimeData()
                    data = mime.data('application/x-qt-windows-mime;value="Csv"')  # csv format from excel
                    data = data.data()  # extract the mime data

                    pasted_data = data.decode('utf-8').split('\n')
                    # strip formatting, remove last entry (which is byte code for excel csv output)
                    formatted_data = [item.strip('\r').split(',') for item in pasted_data][:-1]

                    # for each entry in the formatted data,
                    # enter into appropriate qtablewidget position
                    # e.g., first cell goes in '0, 0' next in '0, 1' etc
                    for row_n, row in enumerate(formatted_data):  # process pasted rows
                        row_num = row_n
                        for col_n, value in enumerate(row):
                            col_num = col_n
                            value_item = QtWidgets.QTableWidgetItem(value)
                            self.setItem(row_num, col_num, value_item)

                    return
                super(customTable, self).keyPressEvent(event)

        # UI object
        NOTNOTALIMS.setObjectName("NOTNOTALIMS")
        NOTNOTALIMS.resize(635, 449)

        # OK/Cancel buttons object
        self.buttonBox = QtWidgets.QDialogButtonBox(NOTNOTALIMS)
        self.buttonBox.setGeometry(QtCore.QRect(340, 380, 241, 61))
        self.buttonBox.setOrientation(QtCore.Qt.Horizontal)
        self.buttonBox.setStandardButtons(QtWidgets.QDialogButtonBox.Cancel | QtWidgets.QDialogButtonBox.Ok)
        self.buttonBox.setCenterButtons(True)
        self.buttonBox.setObjectName("buttonBox")

        # label for data table
        self.data_table_label = QtWidgets.QLabel(NOTNOTALIMS)
        self.data_table_label.setGeometry(QtCore.QRect(20, 20, 221, 21))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.data_table_label.setFont(font)
        self.data_table_label.setObjectName("data_table_label")

        # data table for LC/HC conc and well position
        self.data_table = customTable(NOTNOTALIMS)
        self.data_table.setGeometry(QtCore.QRect(10, 60, 261, 361))
        font = QtGui.QFont()
        font.setFamily("Verdana")
        font.setPointSize(10)
        self.data_table.setFont(font)
        self.data_table.setFrameShape(QtWidgets.QFrame.Panel)
        self.data_table.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.data_table.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarAsNeeded)
        self.data_table.setShowGrid(True)
        self.data_table.setRowCount(97)
        self.data_table.setColumnCount(2)
        self.data_table.setObjectName("data_table")
        item = QtWidgets.QTableWidgetItem()
        self.data_table.setHorizontalHeaderItem(0, item)
        item = QtWidgets.QTableWidgetItem()
        self.data_table.setHorizontalHeaderItem(1, item)
        self.data_table.horizontalHeader().setCascadingSectionResizes(False)
        self.data_table.horizontalHeader().setDefaultSectionSize(114)
        self.data_table.verticalHeader().setVisible(False)

        # line to divide UI into two halves
        self.dividing_line = QtWidgets.QFrame(NOTNOTALIMS)
        self.dividing_line.setGeometry(QtCore.QRect(280, -20, 40, 481))
        self.dividing_line.setLineWidth(4)
        self.dividing_line.setFrameShape(QtWidgets.QFrame.VLine)
        self.dividing_line.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.dividing_line.setObjectName("dividing_line")

        # tab selection object
        # allows switching between the two software functions
        self.tab_selection = QtWidgets.QTabWidget(NOTNOTALIMS)
        self.tab_selection.setGeometry(QtCore.QRect(300, 0, 341, 381))
        self.tab_selection.setObjectName("tab_selection")

        # tab for the hc normalization function
        self.hc_norm_tab = QtWidgets.QWidget()
        self.hc_norm_tab.setObjectName("hc_norm_tab")

        # label for hc normalization tab
        self.hc_norm_label = QtWidgets.QLabel(self.hc_norm_tab)
        self.hc_norm_label.setGeometry(QtCore.QRect(20, 20, 201, 21))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.hc_norm_label.setFont(font)
        self.hc_norm_label.setObjectName("hc_norm_label")

        # label for hc input mass target
        self.hc_input_mass_label = QtWidgets.QLabel(self.hc_norm_tab)
        self.hc_input_mass_label.setGeometry(QtCore.QRect(20, 60, 221, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.hc_input_mass_label.setFont(font)
        self.hc_input_mass_label.setObjectName("hc_input_mass_label")

        # hc input mass target
        self.hc_input_mass = QtWidgets.QDoubleSpinBox(self.hc_norm_tab)
        self.hc_input_mass.setGeometry(QtCore.QRect(240, 60, 61, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(11)
        self.hc_input_mass.setFont(font)
        self.hc_input_mass.setSuffix("")
        self.hc_input_mass.setDecimals(0)
        self.hc_input_mass.setMinimum(100.0)
        self.hc_input_mass.setMaximum(5000.0)
        self.hc_input_mass.setSingleStep(100.0)
        self.hc_input_mass.setProperty("value", 2000.0)
        self.hc_input_mass.setObjectName("hc_input_mass")

        # label for hc remaining volume
        self.vol_remain_label = QtWidgets.QLabel(self.hc_norm_tab)
        self.vol_remain_label.setGeometry(QtCore.QRect(20, 120, 161, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.vol_remain_label.setFont(font)
        self.vol_remain_label.setObjectName("vol_remain_label")

        # hc volume remaining
        self.vol_remain = QtWidgets.QDoubleSpinBox(self.hc_norm_tab)
        self.vol_remain.setGeometry(QtCore.QRect(240, 120, 61, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(11)
        self.vol_remain.setFont(font)
        self.vol_remain.setDecimals(1)
        self.vol_remain.setMinimum(2.5)
        self.vol_remain.setMaximum(60.0)
        self.vol_remain.setSingleStep(0.5)
        self.vol_remain.setProperty("value", 57.0)
        self.vol_remain.setObjectName("vol_remain")

        # label for hc baitset name
        self.baitset_label = QtWidgets.QLabel(self.hc_norm_tab)
        self.baitset_label.setGeometry(QtCore.QRect(20, 180, 121, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.baitset_label.setFont(font)
        self.baitset_label.setObjectName("baitset_label")

        # hc baitset
        self.baitset = QtWidgets.QTextEdit(self.hc_norm_tab)
        self.baitset.setGeometry(QtCore.QRect(240, 180, 61, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.baitset.setFont(font)
        self.baitset.setVerticalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.baitset.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.baitset.setObjectName("baitset")

        # label for LC plate name (for HC norm)
        self.lc_plate_name_norm_label = QtWidgets.QLabel(self.hc_norm_tab)
        self.lc_plate_name_norm_label.setGeometry(QtCore.QRect(20, 240, 221, 21))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.lc_plate_name_norm_label.setFont(font)
        self.lc_plate_name_norm_label.setObjectName("lc_plate_name_norm_label")

        # lc plate name (for HC norm)
        self.lc_plate_name_norm = QtWidgets.QTextEdit(self.hc_norm_tab)
        self.lc_plate_name_norm.setGeometry(QtCore.QRect(20, 281, 281, 41))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.lc_plate_name_norm.setFont(font)
        self.lc_plate_name_norm.setVerticalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.lc_plate_name_norm.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.lc_plate_name_norm.setObjectName("lc_plate_name_norm")

        # add the hc norm tab to selection
        self.tab_selection.addTab(self.hc_norm_tab, "")

        # tab for the mpl function
        self.mpl_tab = QtWidgets.QWidget()
        self.mpl_tab.setObjectName("mpl_tab")

        # mpl parameters label
        self.mpl_params_label = QtWidgets.QLabel(self.mpl_tab)
        self.mpl_params_label.setGeometry(QtCore.QRect(20, 19, 141, 20))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.mpl_params_label.setFont(font)
        self.mpl_params_label.setObjectName("mpl_params_label")

        # label for mpl final conc
        self.mpl_final_conc_label = QtWidgets.QLabel(self.mpl_tab)
        self.mpl_final_conc_label.setGeometry(QtCore.QRect(20, 50, 221, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.mpl_final_conc_label.setFont(font)
        self.mpl_final_conc_label.setObjectName("mpl_final_conc_label")

        # mpl final target conc
        self.mpl_final_conc = QtWidgets.QDoubleSpinBox(self.mpl_tab)
        self.mpl_final_conc.setGeometry(QtCore.QRect(240, 50, 61, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(11)
        self.mpl_final_conc.setFont(font)
        self.mpl_final_conc.setSuffix("")
        self.mpl_final_conc.setMinimum(0.1)
        self.mpl_final_conc.setMaximum(5.0)
        self.mpl_final_conc.setSingleStep(0.1)
        self.mpl_final_conc.setProperty("value", 1.0)
        self.mpl_final_conc.setObjectName("mpl_final_conc")

        # label for mpl pooling volume
        self.sample_draw_vol_label = QtWidgets.QLabel(self.mpl_tab)
        self.sample_draw_vol_label.setGeometry(QtCore.QRect(20, 95, 121, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.sample_draw_vol_label.setFont(font)
        self.sample_draw_vol_label.setObjectName("sample_draw_vol")

        # mpl volume to pool
        self.sample_draw_vol = QtWidgets.QDoubleSpinBox(self.mpl_tab)
        self.sample_draw_vol.setGeometry(QtCore.QRect(240, 95, 61, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(11)
        self.sample_draw_vol.setFont(font)
        self.sample_draw_vol.setDecimals(1)
        self.sample_draw_vol.setMinimum(2.5)
        self.sample_draw_vol.setMaximum(50.0)
        self.sample_draw_vol.setSingleStep(0.1)
        # modify default from 10 -> 4 uL
        self.sample_draw_vol.setProperty("value", 4.0)
        self.sample_draw_vol.setObjectName("sample_draw_vol")

        # label for mpl pooling volume
        self.pool_vol_label = QtWidgets.QLabel(self.mpl_tab)
        self.pool_vol_label.setGeometry(QtCore.QRect(20, 140, 121, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.pool_vol_label.setFont(font)
        self.pool_vol_label.setObjectName("pool_vol_label")

        # mpl volume to pool
        self.pool_vol = QtWidgets.QDoubleSpinBox(self.mpl_tab)
        self.pool_vol.setGeometry(QtCore.QRect(240, 140, 61, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(11)
        self.pool_vol.setFont(font)
        self.pool_vol.setDecimals(1)
        self.pool_vol.setMinimum(2.5)
        self.pool_vol.setMaximum(50.0)
        self.pool_vol.setSingleStep(0.1)
        self.pool_vol.setProperty("value", 7.5)
        self.pool_vol.setObjectName("pool_vol")

        # label for mpl avg frag size
        self.avg_frag_size_label = QtWidgets.QLabel(self.mpl_tab)
        self.avg_frag_size_label.setGeometry(QtCore.QRect(20, 185, 261, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.avg_frag_size_label.setFont(font)
        self.avg_frag_size_label.setObjectName("avg_frag_size_label")

        # mpl avg fragment size
        self.avg_frag_size = QtWidgets.QDoubleSpinBox(self.mpl_tab)
        self.avg_frag_size.setGeometry(QtCore.QRect(240, 185, 61, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(11)
        self.avg_frag_size.setFont(font)
        self.avg_frag_size.setDecimals(0)
        self.avg_frag_size.setMinimum(100.0)
        self.avg_frag_size.setMaximum(1000.0)
        self.avg_frag_size.setSingleStep(1.0)
        self.avg_frag_size.setProperty("value", 300.0)
        self.avg_frag_size.setObjectName("avg_frag_size")

        # label for HC plate name (for MPL)
        self.hc_plate_name_mpl_label = QtWidgets.QLabel(self.mpl_tab)
        self.hc_plate_name_mpl_label.setGeometry(QtCore.QRect(20, 239, 221, 20))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.hc_plate_name_mpl_label.setFont(font)
        self.hc_plate_name_mpl_label.setObjectName("hc_plate_name_mpl_label")

        # HC plate name (for MPL)
        self.hc_plate_name_mpl = QtWidgets.QTextEdit(self.mpl_tab)
        self.hc_plate_name_mpl.setGeometry(QtCore.QRect(20, 280, 281, 40))
        font = QtGui.QFont()
        font.setFamily("Arial")
        font.setPointSize(12)
        self.hc_plate_name_mpl.setFont(font)
        self.hc_plate_name_mpl.setVerticalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.hc_plate_name_mpl.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.hc_plate_name_mpl.setObjectName("hc_plate_name_mpl")

        # add mpl tab to selection
        self.tab_selection.addTab(self.mpl_tab, "")

        self.retranslateUi(NOTNOTALIMS)
        # tab selection index is 0 or 1
        # 0 = HC norm tab
        # 1 = MPL gen tab
        self.tab_selection.setCurrentIndex(1)  # default to MPL creation tab

        # what to do when OK is selected
        # note: if OK pressed but file not saved, script interrupted - will throw error
        def pressOK():
            global notalims_config
            global plate_name
            global sample_data_table
            global worklist_save_path

            sample_data_table = {}
            for row in range(0, 96):  # for all possible rows
                row_data = []
                for col in range(0, 2):  # for all possible columns
                    try:
                        data_item = self.data_table.item(row, col).text()
                        row_data.append(data_item)
                    except Exception:
                        row_data.append(None)
                sample_data_table[row] = row_data

            sample_data_table = pd.DataFrame.from_dict(sample_data_table).T.rename(
                columns={
                    0: 'Well',
                    1: 'Conc'
                }
            ).dropna()

            # note: base path assumes typical user account & directory structure
            base_file_path = f"C:\\Users\\{getpass.getuser()}\\Desktop"

            tab_index = self.tab_selection.currentIndex()

            if tab_index == 0:  # for HC norm
                plate_name = str(self.lc_plate_name_norm.toPlainText())  # includes prefix, e.g., "LC-plate"
                notalims_config = hc_norm(
                    float(round(self.hc_input_mass.value(), 1)),  # target mass
                    float(round(self.vol_remain.value(), 1)),  # lc vol remaining
                    str(self.baitset.toPlainText())  # baitset
                )

                processed_df = process_hc_norm(
                    hc_norm_obj=notalims_config,  # the hc norm config parameters
                    lc_plate_name=plate_name,  # name of input LC plate
                    input_data=sample_data_table  # data from the table widget
                )

                process_log = hc_validate_input(
                    hc_norm_obj=notalims_config,
                    input_data=sample_data_table,
                    output_worklist=processed_df
                )

                # output suggested file name, replace LC with HC
                # for typical file naming convention
                sugg_file_name = f"{plate_name.replace('LC', 'HC')}.csv"

            elif tab_index == 1:  # for MPL gen
                plate_name = str(self.hc_plate_name_mpl.toPlainText())  # includes prefix, e.g., "HC-plate"
                notalims_config = mpl(
                    float(round(self.mpl_final_conc.value(), 3)),  # final conc
                    float(round(self.pool_vol.value(), 3)),  # pool vol
                    float(round(self.avg_frag_size.value(), 1)),  # avg fragment length
                    float(round(self.sample_draw_vol.value(), 1))  # sample draw volume
                )

                processed_df = process_mpl(
                    mpl_obj=notalims_config,  # the mpl config with parameters
                    hc_plate_name=plate_name,  # name of output plate
                    input_data=sample_data_table  # data from the table widget
                )

                process_log = validate_mpl_dilutions(
                    mpl_obj=notalims_config,
                    input_data=sample_data_table,
                    output_worklist=processed_df
                )

                # output suggested file name, replace HC with MPL
                # for typical file naming convention
                sugg_file_name = f"{plate_name.replace('HC', 'MPL')}.csv"

            worklist_save_path = QtWidgets.QFileDialog.getSaveFileName(
                NOTNOTALIMS,
                "Save worklist",
                os.path.join(base_file_path, sugg_file_name),
                "CSV Files (*.csv)"
            )

            worklist_save_path = worklist_save_path[0]  # just want the path

            # save to csv file, ready for instrument to use
            processed_df.to_csv(worklist_save_path, index=None)

            # write a log file to show what was done
            log_file = worklist_save_path.replace('.csv', '.log')
            with open(log_file, 'w') as out_log:
                for item in process_log:
                    out_log.write(item + '\n')

            NOTNOTALIMS.accept()  # closes the app window

            return

        # what to do when cancel is selected
        def pressCancel():
            NOTNOTALIMS.reject()  # closes app winow, no further processing
            return

        self.buttonBox.accepted.connect(pressOK)
        self.buttonBox.rejected.connect(pressCancel)
        QtCore.QMetaObject.connectSlotsByName(NOTNOTALIMS)

    def retranslateUi(self, NOTNOTALIMS):
        _translate = QtCore.QCoreApplication.translate
        NOTNOTALIMS.setWindowTitle(_translate("NOTNOTALIMS", f"NOTNOTALIMS GUI {script_vers}"))
        item = self.data_table.horizontalHeaderItem(0)
        item.setText(_translate("NOTNOTALIMS", "Source Well"))
        item = self.data_table.horizontalHeaderItem(1)
        item.setText(_translate("NOTNOTALIMS", "Conc (ng/μL)"))
        self.data_table_label.setText(_translate("NOTNOTALIMS", "Paste conc values here:"))
        self.lc_plate_name_norm.setHtml(_translate(
            "NOTNOTALIMS", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
            "<html><head><meta name=\"qrichtext\" content=\"1\" /><style type=\"text/css\">\n"
            "p, li { white-space: pre-wrap; }\n"
            "</style></head><body style=\" font-family:\'Arial\'; font-size:12pt; font-weight:400; font-style:normal;\">\n"
            "<p style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\">LC-</p></body></html>"))
        self.hc_norm_label.setText(_translate("NOTNOTALIMS", "HC norm parameters:"))
        self.hc_input_mass_label.setText(_translate("NOTNOTALIMS", "Target input mass (ng)"))
        self.vol_remain_label.setText(_translate("NOTNOTALIMS", "LC remaining vol (μL)"))
        self.baitset.setHtml(_translate(
            "NOTNOTALIMS", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
            "<html><head><meta name=\"qrichtext\" content=\"1\" /><style type=\"text/css\">\n"
            "p, li { white-space: pre-wrap; }\n"
            "</style></head><body style=\" font-family:\'Arial\'; font-size:12pt; font-weight:400; font-style:normal;\">\n"
            "<p style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\">AB1</p></body></html>"))
        self.baitset_label.setText(_translate("NOTNOTALIMS", "Baitset"))
        self.lc_plate_name_norm_label.setText(_translate("NOTNOTALIMS", "LC plate name:"))
        self.tab_selection.setTabText(self.tab_selection.indexOf(self.hc_norm_tab), _translate("NOTNOTALIMS", "HC normalization"))
        self.hc_plate_name_mpl.setHtml(_translate(
            "NOTNOTALIMS", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
            "<html><head><meta name=\"qrichtext\" content=\"1\" /><style type=\"text/css\">\n"
            "p, li { white-space: pre-wrap; }\n"
            "</style></head><body style=\" font-family:\'Arial\'; font-size:12pt; font-weight:400; font-style:normal;\">\n"
            "<p style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\">HC-</p></body></html>"))
        self.avg_frag_size_label.setText(_translate("NOTNOTALIMS", "Avg. fragment size (bp)"))
        self.pool_vol_label.setText(_translate("NOTNOTALIMS", "Vol to pool (μL)"))
        self.sample_draw_vol_label.setText(_translate("NOTNOTALIMS", "Vol to draw (μL)"))
        self.mpl_final_conc_label.setText(_translate("NOTNOTALIMS", "Target pool conc (nM)"))
        self.hc_plate_name_mpl_label.setText(_translate("NOTNOTALIMS", "HC plate name:"))
        self.mpl_params_label.setText(_translate("NOTNOTALIMS", "MPL parameters:"))
        self.tab_selection.setTabText(self.tab_selection.indexOf(self.mpl_tab), _translate("NOTNOTALIMS", "MPL generation"))


if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    NOTNOTALIMS = QtWidgets.QDialog()
    ui = Ui_NOTNOTALIMS()
    ui.setupUi(NOTNOTALIMS)
    NOTNOTALIMS.show()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\rratnappan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
%run C:/Users/rratnappan/Documents/Confera/Hamilton_worklist/notnotalims.py
